# ANALYSER

In [22]:
import Modele
from Variables import *
import csv
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import time
import statistics

In [23]:
# Idée de test 1 : nbr de voiture total sur la route hors fin


def nbr_voiture(route, direction, traffic, duree_feu, duree_test):

    for i in range(len(route)):
        for j in range(len(route[i])):
            if route[i][j] != 0 and route[i][j][0] == "Feu":
                route[i][j][2] = duree_feu

    res = []

    for temps in range(duree_test):

        route, direction, traffic = Modele.mouvement(route, direction, traffic, temps)

        compt = 0

        for i in range(len(traffic)):
            for j in range(len(traffic[i])):
                if isinstance(traffic[i][j], list):
                    if route[i][j] != 0 and route[i][j][0] == "Route":
                        compt += traffic[i][j].count(1)
                    elif route[i][j] != 0 and route[i][j][0] != "Fin":
                        compt += traffic[i][j][0]

        res.append(compt)

    return res


def test_1(route, direction, traffic, duree_min, duree_max, nbr_test, nbr_ite):

    res = []
    
    for duree in tqdm(range(duree_min, duree_max)):
        for i in range(nbr_test):
            compt = []
            compt.append(sum(nbr_voiture(route, direction, traffic, duree, nbr_ite)))
        res.append(statistics.mean(compt))
        
    return res


r = test_1(route_etude, direction_etude, traffic_etude, 40, 120, 10, 600)
index = sorted(range(len(r)), key=lambda i: r[i])

print(r)
print([i + 40 for i in index])

100%|██████████| 80/80 [00:41<00:00,  1.91it/s]

[5960, 5746, 5633, 5715, 6107, 6574, 6516, 5446, 5145, 6671, 6085, 5712, 7318, 6484, 7247, 6321, 5039, 6690, 6251, 6720, 6806, 7495, 5703, 6896, 7298, 8049, 7223, 7705, 6917, 7301, 6753, 6797, 7731, 7480, 6937, 8088, 5352, 7574, 8193, 9731, 6553, 8657, 7486, 9055, 7024, 9994, 7113, 9017, 8562, 7651, 9384, 6997, 7108, 7884, 9033, 7134, 10498, 7953, 10479, 9621, 7453, 7580, 10257, 9334, 6894, 8855, 8669, 8500, 8125, 8724, 9606, 9237, 10540, 9370, 10534, 10193, 10225, 11819, 13517, 12819]
[56, 48, 76, 47, 42, 62, 51, 43, 41, 40, 50, 44, 58, 55, 53, 46, 80, 45, 49, 57, 59, 70, 71, 60, 104, 63, 68, 74, 91, 84, 92, 86, 95, 66, 54, 64, 69, 52, 100, 73, 82, 61, 77, 101, 89, 67, 72, 93, 97, 65, 75, 108, 78, 107, 88, 81, 106, 109, 105, 87, 94, 83, 111, 103, 113, 90, 110, 99, 79, 85, 115, 116, 102, 98, 96, 114, 112, 117, 119, 118]


Résultat : Test naif qui a pour objectif de vider au plus vite les routes sans vraiment vouloir minimiser les embouteillage. On obtient en général un temps optimal le plus faible étrange !

In [24]:
# Test 2 : Nombre de voiture sur les routes = embouteillage
# On veut que le phenomène soit penalisé x2 si plus de la moitié de la route occupé par des voitures
# Et x5 les voitures sur les cases départ avec plus de 1 voiture

def nbr_voiture_coef(route, direction, traffic, duree_feu, duree_test):

    for i in range(len(route)):
        for j in range(len(route[i])):
            if route[i][j] != 0 and route[i][j][0] == "Feu":
                route[i][j][2] = duree_feu

    res = []

    for temps in range(duree_test):

        route, direction, traffic = Modele.mouvement(route, direction, traffic, temps)

        compt = 0

        for i in range(len(traffic)):
            for j in range(len(traffic[i])):
                if isinstance(traffic[i][j], list):
                    if route[i][j] != 0 and route[i][j][0] == "Route":
                        if traffic[i][j].count(1) > len(route[i][j])/2:
                            compt += 5*traffic[i][j].count(1)
                        else:
                            compt += traffic[i][j].count(1)
                    elif route[i][j] != 0 and route[i][j][0] == "Départ":
                        if traffic[i][j][0] > 1:
                            compt += 10*traffic[i][j][0]
                        else:
                            compt += traffic[i][j][0]
                    elif route[i][j] != 0 and route[i][j][0] != "Fin":
                        compt += traffic[i][j][0]

        res.append(compt)

    return res

def test_2(route, direction, traffic, duree_min, duree_max, nbr_test, nbr_ite):

    res = []
    
    for duree in tqdm(range(duree_min, duree_max)):
        for i in range(nbr_test):
            compt = []
            compt.append(sum(nbr_voiture_coef(route, direction, traffic, duree, nbr_ite)))
        res.append(statistics.mean(compt))
        
    return res

r = test_2(route_etude, direction_etude, traffic_etude, 40, 120, 10, 600)
index = sorted(range(len(r)), key=lambda i: r[i])

print(r)
print([i + 40 for i in index])

100%|██████████| 80/80 [00:44<00:00,  1.80it/s]

[25329, 16696, 25158, 27874, 24991, 22309, 31698, 25142, 26107, 25660, 22818, 19332, 27225, 31793, 32524, 28343, 25407, 24556, 29976, 33542, 27314, 29655, 38574, 31107, 26285, 27665, 36269, 29759, 25230, 27566, 40728, 29624, 39517, 25713, 28886, 38451, 41908, 30268, 27635, 37207, 32896, 33032, 33534, 42184, 33700, 43241, 34764, 28678, 28405, 32184, 30389, 26627, 37811, 37808, 38226, 30103, 31187, 31992, 40391, 36015, 36226, 34287, 40884, 35830, 45913, 35841, 37982, 44734, 41541, 42471, 40421, 38899, 38406, 41540, 45099, 43150, 43723, 52985, 55133, 50429]
[41, 51, 45, 50, 57, 44, 47, 42, 68, 40, 56, 49, 73, 48, 64, 91, 52, 60, 69, 78, 65, 43, 55, 88, 87, 74, 71, 61, 67, 58, 95, 77, 90, 63, 96, 46, 53, 97, 89, 54, 80, 81, 82, 59, 84, 101, 86, 103, 105, 99, 100, 66, 79, 93, 92, 106, 94, 112, 75, 62, 111, 72, 98, 110, 70, 102, 113, 108, 76, 83, 109, 115, 85, 116, 107, 114, 104, 119, 117, 118]
